# Projeto SuperComputação 2023.2

## Aluno: Guilherme Fontana Louro

# Introdução do problema

## Análise de Redes Sociais: encontrando a clique máxima em um grafo.

![Exemplo Clique](https://upload.wikimedia.org/wikipedia/commons/thumb/d/d0/VR_complex.svg/1200px-VR_complex.svg.png)


A análise de redes sociais (ARS) é uma abordagem oriunda de áreas tais como Sociologia, Psicologia Social e Antropologia. Tal abordagem estuda as ligações relacionais (*relational tie*) entre atores sociais. Os atores na ARS podem ser tanto pessoas e empresas, analisadas como unidades individuais, quanto unidades sociais coletivas como, por exemplo, departamentos dentro de uma organização, agências de serviço público em uma cidade, estados-nações de um continente, dentre outras. A ARS difere fundamentalmente de outros estudos pelo fato de que sua ênfase não é nos atributos (características) dos atores, mas nas ligações entre eles.


A idéia de uma clique em um grafo é relativamente simples. No nível mais geral, uma clique é um subconjunto de uma rede no qual os atores são mais próximos entre si do que com outros membros da rede. Em termo de laços de amizade, por exemplo, não é incomum encontrar grupos humanos que formam cliques baseando-se em idade, gênero, raça, etnia, religião, ideologia, e muitas coisas coisas. Uma clique é, portanto, um conjunto de vértices em um grafo em que cada par de vértices está diretamente conectado por uma aresta.

Encontrar a clique máxima em um grafo é uma tarefa computacionalmente desafiadora devido à natureza combinatória do problema.  A dificuldade computacional surge da necessidade de explorar todas as combinações possíveis de vértices para identificar a maior clique, o que se torna exponencial em relação ao número de vértices. Isso resulta em uma complexidade computacional alta, mesmo para grafos moderadamente grandes.

A importância de estudar cliques está notavelmente presente na análise de redes sociais, onde as cliques representam grupos coesos de indivíduos que compartilham interesses, amizades ou conexões em comum. A identificação de cliques ajuda a entender a estrutura de uma rede social, identificar influenciadores e grupos de afinidade, além de auxiliar na detecção de comunidades e na análise de dinâmicas sociais.

As cliques são importantes, pois além de desenvolver em seus membros comportamentos homogêneos, elas têm, por definição, grande proximidade, aumentando a velocidade das trocas. Assim, informações dirigidas a uma clique são rapidamente absorvidas pelos seus membros, que tendem a percebê-las de forma semelhante. Isso é importante, por exemplo, em estratégias de segmentação.


Portanto, a resolução eficiente do problema da clique máxima tem aplicações valiosas em áreas que vão desde a ciência da computação até a análise de dados em redes sociais.

1. **Abordagem Exaustiva**

A exaustão é uma abordagem que seleciona iterativamente os vértices para formar um clique, geralmente começando com um vértice e adicionando outros que tenham o maior número de vizinhos já na clique. Aqui está um pseudo-código simplificado para detectar cliques em um grafo usando essa abordagem. ATENÇÃO: esse pseudo-codigo **não** é a solução completa dessa abordagem. Você pode se inspirar nele para compreender como resolver o problema, mas é parte de sua tarefa desenvolver a solução.

Implemente o código em C++.

Aproveite para pensar se é possível adotar alguma heurística. Por exemplo, ordenar os nós em função do grau de adjacência, ajuda?


**Sua segunda tarefa**: Até qual tamanho de problema você conseguiu obter um resultado em tempo hábil (aprox. 15 min)? Você deve apresentar um pseudo-codigo de uma heurística para otimizar essa tarefa exaustiva. É possível implementar alguma poda? Algum critério que evite calcular um nó, dado que você já descobriu uma clique maior?

In [ ]:
#include <algorithm>
#include <fstream>
#include <iostream>
#include <omp.h>
#include <vector>
#include "ReadGraph.cpp"

// Function to check if a node belongs to the clique
std::tuple<std::vector<int>, std::vector<int>> CliqueCheck(
    const std::vector<std::vector<int>> &graph,
    const std::vector<int> &candidates,
    const std::vector<int> &currentClique,
    int vertex) {
    std::vector<int> newCandidates = candidates;
    newCandidates.erase(std::remove(newCandidates.begin(), newCandidates.end(), vertex), newCandidates.end());

    bool canAdd = true;
    for (int u : currentClique) {
        if (graph[u][vertex] == 0) {
            canAdd = false;
            break;
        }
    }

    std::vector<int> newClique = currentClique;
    if (canAdd) {
        newClique.push_back(vertex);

        std::vector<int> updatedCandidates;
        for (int u : newCandidates) {
            bool adjacentToAll = true;
            for (int c : newClique) {
                if (graph[u][c] == 0) {
                    adjacentToAll = false;
                    break;
                }
            }
            if (adjacentToAll) {
                updatedCandidates.push_back(u);
            }
        }
        newCandidates = updatedCandidates;
    }

    return std::make_tuple(newClique, newCandidates);
}

// Function to find the maximum clique in a graph
std::vector<int> FindMaximumClique(
    const std::vector<std::vector<int>> &graph,
    const std::vector<int> &candidates,
    const std::vector<int> &currentClique) {
    std::vector<int> maximumClique = currentClique;

    for (int candidate : candidates) {
        std::vector<int> newCandidates;
        std::vector<int> newClique;
        std::tie(newClique, newCandidates) = CliqueCheck(graph, candidates, currentClique, candidate);

        if (!newCandidates.empty()) {
            newClique = FindMaximumClique(graph, newCandidates, newClique);
        }

        if (newClique.size() >= maximumClique.size()) {
            maximumClique = newClique;
        }
    }

    return maximumClique;
}

int main() {
    int numVertices = 45;
    std::vector<std::vector<int>> graph;
    graph = ReadGraph("grafo.txt", numVertices);

    std::vector<int> allVertices;
    for (int i = 0; i < numVertices; ++i) {
        allVertices.push_back(i);
    }

    std::vector<int> initialClique;

    double startTime = omp_get_wtime();

    std::vector<int> maximumClique = FindMaximumClique(graph, allVertices, initialClique);

    double duration = omp_get_wtime() - startTime;

    std::cout << "Maximum clique: ";
    for (int u : maximumClique) {
        std::cout << u + 1 << " ";
    }
    std::cout << std::endl;

    std::cout << "Duration: " << duration << " s";

    return 0;
}


## Sua segunda tarefa: 
Até qual tamanho de problema você conseguiu obter um resultado em tempo hábil (aprox. 15 min)? Você deve apresentar um pseudo-codigo de uma heurística para otimizar essa tarefa exaustiva. É possível implementar alguma poda? Algum critério que evite calcular um nó, dado que você já descobriu uma clique maior?

## Resposta:
Consegui resolver o problema para grafos de até 45 vértices em tempo hábil, no caso, 316s. Para grafos maiores, o tempo de execução é muito alto, inviabilizando a resolução do problema.
O resultado desse teste pode ser visto na imagem abaixo, que mostra o tempo de execução para grafos de 45 vértices e demonstra que o resultado é obtido em menos de 15 minutos e é igual ao retornado pelo arquivo fornecido verifica_max.py.

![Resultado](imgs/resultado.png)

Sobre a implementação de alguma poda, é possível realizar uma técnica conhecida como poda por limites(pruining by bounds), que consiste em utilizar o tamanho da maior clique já encontrada. Se o tamanho da clique atual, somado ao número de vértices restantes, for menor que o tamanho da maior clique já encontrada, então não é necessário continuar a busca, pois não será possível encontrar uma clique maior que a já encontrada.

A poda por limites ajuda a reduzir o número de ramos explorados, economizando tempo computacional.


### Pseudo-código da solução:

```
Função EncontrarCliqueMaxima(grafo, numVertices)
    cliqueMaxima = ListaVazia()
    candidatos = ListaDeNós()  # Inicialmente, todos os nós são candidatos

    Para cada i de 0 até numVertices - 1 Faça
        Adicione i à lista de candidatos

    Enquanto candidatos não estiver vazia Faça
        v = Último elemento de candidatos
        Remova o último elemento de candidatos

        podeAdicionar = Verdadeiro

        Para cada u em cliqueMaxima Faça
            Se grafo[u][v] == 0 Então
                podeAdicionar = Falso
                Pare o loop
            Fim Se
        Fim Para

        Se podeAdicionar for Verdadeiro Então
            Adicione v a cliqueMaxima
            novosCandidatos = ListaDeNós()

            Para cada u em candidatos Faça
                adjacenteATodos = Verdadeiro

                Para cada c em cliqueMaxima Faça
                    Se grafo[u][c] == 0 Então
                        adjacenteATodos = Falso
                        Pare o loop
                    Fim Se
                Fim Para

                Se adjacenteATodos for Verdadeiro Então
                    Adicione u a novosCandidatos
                Fim Se
            Fim Para

            # Poda por limites
            Se tamanho(cliqueMaxima) + tamanho(novosCandidatos) >= tamanho(cliqueMaxima) Então
                candidatos = novosCandidatos
            Senão
                Parar a exploração desse ramo
            Fim Se
        Fim Se
    Fim Enquanto

    Retorne cliqueMaxima
Fim Função
```

## Implementação

Na célula abaixo, foi implementado o código em C++ para a solução do problema com a implementação da poda por limites.

In [ ]:
#include <iostream>
#include <vector>
#include <fstream>
#include <algorithm>
#include <omp.h>

// Function to read the graph from the input file
std::tuple<std::vector<std::vector<int>>, int> ReadGraph(const std::string& filename) {
    std::ifstream file(filename);
    int numVertices, numEdges;
    file >> numVertices >> numEdges;

    std::vector<std::vector<int>> graph(numVertices, std::vector<int>(numVertices, 0));

    for (int i = 0; i < numEdges; ++i) {
        int u, v;
        file >> u >> v;
        graph[u - 1][v - 1] = 1;
        graph[v - 1][u - 1] = 1;  // Undirected graph
    }

    file.close();

    return std::make_tuple(graph, numVertices);
}

// Function to check if a vertex belongs to the clique
std::tuple<std::vector<int>, std::vector<int>> CliqueCheck(
    const std::vector<std::vector<int>>& graph,
    const std::vector<int>& candidates,
    const std::vector<int>& currentClique,
    int vertex
) {
    std::vector<int> newCandidates = candidates;
    newCandidates.erase(std::remove(newCandidates.begin(), newCandidates.end(), vertex), newCandidates.end());

    bool canAdd = true;
    for (int u : currentClique) {
        if (graph[u][vertex] == 0) {
            canAdd = false;
            break;
        }
    }

    std::vector<int> newClique = currentClique;
    if (canAdd) {
        newClique.push_back(vertex);

        std::vector<int> updatedCandidates;
        for (int u : newCandidates) {
            bool adjacentToAll = true;
            for (int c : newClique) {
                if (graph[u][c] == 0) {
                    adjacentToAll = false;
                    break;
                }
            }
            if (adjacentToAll) {
                updatedCandidates.push_back(u);
            }
        }
        newCandidates = updatedCandidates;
    }

    return std::make_tuple(newClique, newCandidates);
}

// Function to find the maximum clique in a graph with pruning by bounds
std::vector<int> FindMaximumCliqueWithPruining(
    const std::vector<std::vector<int>>& graph,
    const std::vector<int>& candidates,
    const std::vector<int>& currentClique,
    std::vector<int>& maximumClique
) {
    if (candidates.empty()) {
        if (currentClique.size() > maximumClique.size()) {
            maximumClique = currentClique;
        }
        return maximumClique;
    }

    // Poda por limites
    if (currentClique.size() + candidates.size() <= maximumClique.size()) {
        return maximumClique;
    }

    for (int candidate : candidates) {
        std::vector<int> newCandidates;
        std::vector<int> newClique;
        std::tie(newClique, newCandidates) = CliqueCheck(graph, candidates, currentClique, candidate);

        if (!newCandidates.empty()) {
            newClique = FindMaximumCliqueWithPruining(graph, newCandidates, newClique, maximumClique);
        }

        if (newClique.size() >= maximumClique.size()) {
            maximumClique = newClique;
        }
    }

    return maximumClique;
}

int main() {
    int numVertices;
    std::vector<std::vector<int>> graph;
    std::tie(graph, numVertices) = ReadGraph("grafo_gerado.txt");
    std::cout << "Number of vertices: " << numVertices << std::endl;

    std::vector<int> allVertices;
    for (int i = 0; i < numVertices; ++i) {
        allVertices.push_back(i);
    }

    std::vector<int> initialClique;
    std::vector<int> maximumClique;

    double startTime = omp_get_wtime();

    maximumClique = FindMaximumCliqueWithPruining(graph, allVertices, initialClique, maximumClique);

    double duration = omp_get_wtime() - startTime;

    // SORT AND PRINT
    std::sort(maximumClique.begin(), maximumClique.end());
    std::cout << "Maximum clique found of size " << maximumClique.size() << ": [";
    for (size_t i = 0; i < maximumClique.size(); ++i) {
        std::cout << maximumClique[i] + 1;
        if (i < maximumClique.size() - 1) {
            std::cout << ", ";
        }
    }
    std::cout << "]" << std::endl;

    std::cout << "Duration: " << duration << " s";

    return 0;
}


2. **Implementação com Threads OpenMP**

A implementação de uma solução com OpenMP para encontrar cliques em um grafo usando a heurística gulosa envolve a paralelização das iterações do algoritmo em diferentes threads. O OpenMP simplifica a criação de threads e a coordenação entre elas.


Nessa implementação você deve apresentar o código-fonte modificado, justificar a alteração feita, e mostrar também o speed up obtido. Faça uso de gráficos e tabelas para mostrar que sua implementação em openMP valeu a pena. Observe que mesmo assim você não vai conseguir resolver exaustivamente o problema, mas o tamanho do grafo será maior, e o tempo de processamento para um mesmo tamanho deve ser preferencialmente menor.

In [ ]:
#include "ReadGraph.cpp"
#include <algorithm>
#include <fstream>
#include <iostream>
#include <omp.h>
#include <tuple>
#include <vector>

// Function to check if a node belongs to the clique
std::tuple<std::vector<int>, std::vector<int>> CliqueCheck(
    const std::vector<std::vector<int>> &graph,
    const std::vector<int> &candidates,
    const std::vector<int> &currentClique,
    int vertex) {
    std::vector<int> newCandidates = candidates;
    newCandidates.erase(std::remove(newCandidates.begin(), newCandidates.end(), vertex), newCandidates.end());

    bool canAdd = true;
    for (int u : currentClique) {
        if (graph[u][vertex] == 0) {
            canAdd = false;
            break;
        }
    }

    std::vector<int> newClique = currentClique;
    if (canAdd) {
        newClique.push_back(vertex);

        std::vector<int> updatedCandidates;
        #pragma omp parallel for
        for (int u : newCandidates) {
            bool adjacentToAll = true;
            for (int c : newClique) {
                if (graph[u][c] == 0) {
                    adjacentToAll = false;
                    break;
                }
            }
            if (adjacentToAll) {
                #pragma omp critical
                updatedCandidates.push_back(u);
            }
        }
        newCandidates = updatedCandidates;
    }

    return std::make_tuple(newClique, newCandidates);
}

// Function to find the maximum clique in a graph
std::vector<int> FindMaximumClique(
    const std::vector<std::vector<int>> &graph,
    const std::vector<int> &candidates,
    const std::vector<int> &currentClique) {
    std::vector<int> maximumClique = currentClique;

    #pragma omp parallel for
    for (int candidate : candidates) {
        std::vector<int> newCandidates;
        std::vector<int> newClique;
        std::tie(newClique, newCandidates) = CliqueCheck(graph, candidates, currentClique, candidate);

        if (!newCandidates.empty()) {
            newClique = FindMaximumClique(graph, newCandidates, newClique);
        }

        #pragma omp critical
        if (newClique.size() >= maximumClique.size()) {
            maximumClique = newClique;
        }
    }

    return maximumClique;
}

int main() {
    int numVertices = 50;
    std::vector<std::vector<int>> graph;
    graph = ReadGraph("grafo.txt", numVertices);

    std::vector<int> allVertices;
    for (int i = 0; i < numVertices; ++i) {
        allVertices.push_back(i);
    }

    std::vector<int> initialClique;

    double startTime = omp_get_wtime();

    std::vector<int> maximumClique = FindMaximumClique(graph, allVertices, initialClique);

    double duration = omp_get_wtime() - startTime;

    std::cout << "Maximum clique: ";
    for (int u : maximumClique) {
        std::cout << u + 1 << " ";
    }
    std::cout << std::endl;

    std::cout << "Duration: " << duration << " s";

    return 0;
}



3. **Paralelização e Distribuição do Processamento com MPI**

Nesta implementação, você deve dividir o problema em várias partes e distribuí-las para diferentes processadores usando a biblioteca MPI (*Message Passing Interface*). Cada processador será responsável por encontrar cliques em uma parte do grafo, e os resultados serão combinados no final para encontrar todas as cliques no grafo.

In [ ]:
#include <algorithm>
#include <fstream>
#include <iostream>
#include <mpi.h>
#include <tuple>
#include <vector>
#include "ReadGraph.cpp"

// Function to check if a node belongs to the clique
std::tuple<std::vector<int>, std::vector<int>> CliqueCheck(
    const std::vector<std::vector<int>> &graph,
    const std::vector<int> &candidates,
    const std::vector<int> &currentClique,
    int vertex) {
    std::vector<int> newCandidates = candidates;
    newCandidates.erase(std::remove(newCandidates.begin(), newCandidates.end(), vertex), newCandidates.end());

    bool canAdd = true;
    for (int u : currentClique) {
        if (graph[u][vertex] == 0) {
            canAdd = false;
            break;
        }
    }

    std::vector<int> newClique = currentClique;
    if (canAdd) {
        newClique.push_back(vertex);

        std::vector<int> updatedCandidates;
        for (int u : newCandidates) {
            bool adjacentToAll = true;
            for (int c : newClique) {
                if (graph[u][c] == 0) {
                    adjacentToAll = false;
                    break;
                }
            }
            if (adjacentToAll) {
                updatedCandidates.push_back(u);
            }
        }
        newCandidates = updatedCandidates;
    }

    return std::make_tuple(newClique, newCandidates);
}

// Function to find the maximum clique in a graph
std::vector<int> FindMaximumClique(
    const std::vector<std::vector<int>> &graph,
    const std::vector<int> &candidates,
    const std::vector<int> &currentClique) {
    std::vector<int> maximumClique = currentClique;

    for (int candidate : candidates) {
        std::vector<int> newCandidates;
        std::vector<int> newClique;
        std::tie(newClique, newCandidates) = CliqueCheck(graph, candidates, currentClique, candidate);

        if (!newCandidates.empty()) {
            newClique = FindMaximumClique(graph, newCandidates, newClique);
        }

        if (newClique.size() >= maximumClique.size()) {
            maximumClique = newClique;
        }
    }

    return maximumClique;
}

int main(int argc, char *argv[]) {
    MPI_Init(&argc, &argv);

    int rank, size;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    if (size != 2) {
        std::cerr << "This example assumes 2 MPI processes." << std::endl;
        MPI_Abort(MPI_COMM_WORLD, 1);
    }

    int numVertices = 50;
    std::vector<std::vector<int>> graph;
    std::vector<int> allVertices;
    std::vector<int> initialClique;

    if (rank == 0) {
        graph = ReadGraph("grafo.txt", numVertices);

        for (int i = 0; i < numVertices; ++i) {
            allVertices.push_back(i);
        }
    }

    MPI_Bcast(&numVertices, 1, MPI_INT, 0, MPI_COMM_WORLD);

    if (rank == 1) {
        graph.resize(numVertices, std::vector<int>(numVertices, 0));
    }

    MPI_Bcast(graph.data(), numVertices * numVertices, MPI_INT, 0, MPI_COMM_WORLD);

    double startTime = MPI_Wtime();

    std::vector<int> localClique = FindMaximumClique(graph, allVertices, initialClique);

    double duration = MPI_Wtime() - startTime;

    if (rank == 0) {
        std::vector<int> remoteClique(size);

        MPI_Recv(remoteClique.data(), size, MPI_INT, 1, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);

        // Combine results from different processes
        if (remoteClique.size() > localClique.size()) {
            localClique = remoteClique;
        }

        std::cout << "Maximum clique: ";
        for (int u : localClique) {
            std::cout << u + 1 << " ";
        }
        std::cout << std::endl;

        std::cout << "Duration: " << duration << " s" << std::endl;
    } else if (rank == 1) {
        MPI_Send(localClique.data(), localClique.size(), MPI_INT, 0, 0, MPI_COMM_WORLD);
    }

    MPI_Finalize();

    return 0;
}


## Como apresentar seus resultados


Você deverá focar em comparar suas implementações em relação ao speedup. Aumente o tamanho do grafo e das arestas, e busque determinar a clique máxima.

## Códigos-fonte de apoio

#### Códigos fornecidos pelos professores para auxiliar na implementação


1. Código-fonte de geração do grafo (em Python)


```python

import networkx as nx
import random

# Parâmetros
num_vertices = 1000  # Número de vértices no grafo
probabilidade_conexao = 0.7  # Probabilidade de haver uma aresta entre dois vértices (ajuste conforme necessário)

# Crie um grafo aleatório densamente conectado
grafo = nx.fast_gnp_random_graph(num_vertices, probabilidade_conexao)

# Nome do arquivo de saída
nome_arquivo = "grafo.txt"

# Abra o arquivo para escrita
with open(nome_arquivo, 'w') as arquivo:
    # Escreva a quantidade de vértices e número de arestas na primeira linha
    arquivo.write(f"{num_vertices} {grafo.number_of_edges()}\n")

    # Escreva as arestas no formato de lista de adjacência
    for aresta in grafo.edges():
        arquivo.write(f"{aresta[0]+1} {aresta[1]+1}\n")  # +1 para ajustar os índices (começando em 1)

print(f"Grafo densamente conectado gerado e salvo em '{nome_arquivo}'.")

```


2. Trecho de código-fonte para leitura do grafo e armazenamento como matriz de adjacência

```cpp
// Função para ler o grafo a partir do arquivo de entrada
std::vector<std::vector<int>> LerGrafo(const std::string& nomeArquivo, int& numVertices) {
    std::ifstream arquivo(nomeArquivo);
    int numArestas;
    arquivo >> numVertices >> numArestas;

    std::vector<std::vector<int>> grafo(numVertices, std::vector<int>(numVertices, 0));

    for (int i = 0; i < numArestas; ++i) {
        int u, v;
        arquivo >> u >> v;
        grafo[u - 1][v - 1] = 1;
        grafo[v - 1][u - 1] = 1;  // O grafo é não direcionado
    }

    arquivo.close();

    return grafo;
}
```

3. Verificar (em Python) se seu programa encontrou a clique correta. Para isso, use a implementação abaixo, ela já está adaptada para ler nosso arquivo de input, ignorando a primeira linha.


```python
import networkx as nx

# Nome do arquivo de entrada
nome_arquivo = "seu_arquivo.txt"

# Abrir o arquivo e pular a primeira linha
with open(nome_arquivo, 'r') as arquivo:
    next(arquivo)  # Pula a primeira linha

    # Lê o grafo a partir das linhas restantes
    G = nx.parse_adjlist(arquivo)

# Encontrar todas as cliques maximais
cliques_maximais = list(nx.find_cliques(G))

# Encontrar a clique máxima (a maior)
clique_maxima = max(cliques_maximais, key=len)

print("Cliques maximais encontradas:")
for clique in cliques_maximais:
    print(clique)

print("Clique máxima encontrada:", clique_maxima)


```
